<a href="https://colab.research.google.com/github/sunghwan-moon/2021_capstone_BrainAgePrediction_by_Federated_Learning/blob/main/Local_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#!pip install tensorflow_federated
#!pip install nilearn
#!pip install transformations
#!pip install nest_asyncio

In [2]:
import collections
import numpy as np
import tensorflow as tf
import threading
from random import gauss
from scipy.ndimage.interpolation import map_coordinates
import operator
from scipy.ndimage.interpolation import shift,rotate
import pandas as pd
import os
import scipy.ndimage as nd
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score,mean_absolute_error
import numpy as np
import matplotlib.pyplot as plt
import math
import sys
import random
import glob
import nibabel as nib
from collections import defaultdict
from tensorflow.keras.utils import Sequence
import nest_asyncio
nest_asyncio.apply()
#from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Input, add, concatenate,GlobalAveragePooling3D,Conv3D,LeakyReLU,ELU, MaxPooling3D,AveragePooling3D
from tensorflow.keras.regularizers import l2 as L2
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from tensorflow.keras.optimizers import Adam, SGD,Adagrad
from keras.engine.topology import Layer
from keras.layers import BatchNormalization
from keras.layers import Input, InputLayer, InputSpec
#from tensorflow.keras import backend as K
#from tensorflow.keras.callbacks import TensorBoard, EarlyStopping
#import tensorflow.compat.v1 as tf
# tf.compat.v1.enable_v2_behavior()
# tf.compat.v1.enable_eager_execution()

from tensorflow.compat.v1 import keras
from tensorflow.compat.v1.keras.layers import Dense, Dropout, Activation, Flatten, Input, add, concatenate,GlobalAveragePooling3D,Conv3D,LeakyReLU,ELU, MaxPooling3D,AveragePooling3D
from tensorflow.keras import layers

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
def getData(path):
  original_data = pd.read_excel(path)
  original_data.dropna(inplace=True)
  original_data.drop(["ID","Sex"], axis = 1, inplace=True)

  return original_data

In [5]:
data = getData('/content/drive/MyDrive/소융_캡스톤/뇌 MRI 사진_교수님/IXI_train_test_age.xlsx')

In [6]:
train, test = train_test_split(data, test_size = 0.2, random_state =1000)

In [7]:
x_train = train.drop(["Age"],axis=1).to_numpy(np.float32)
y_train = train["Age"].to_numpy(np.float32)


x_test = test.drop(["Age"],axis=1).to_numpy(np.float32)
y_test = test["Age"].to_numpy(np.float32)


num_train_image = len(x_train)
num_test_image= len(x_test)
print("Train set: ", num_train_image)
print("Test set: ", num_test_image)

Train set:  370
Test set:  93


In [8]:
x_train.shape

(370, 69)

In [9]:
from sklearn.svm import SVR


In [10]:
def build_model():
  model = keras.Sequential([
   layers.Dense(69, activation='relu', input_shape=(69,)),
   layers.Dense(69, activation='relu'),
   layers.Dense(1)
  ])

  optimizer = tf.keras.optimizers.SGD(0.01)

  model.compile(loss='mae',
               optimizer=optimizer,
               metrics=['mae'])
  return model

In [11]:
def build_model():
  model = keras.Sequential([
   layers.Dense(1, input_shape=(69,), activation = 'linear'),
   layers.Dense(10),
   layers.Dense(5),
   layers.Dense(1)
  ])

  optimizer = tf.keras.optimizers.Adam(0.001)

  model.compile(loss='mae',
               optimizer=optimizer,
               metrics=['mae'])

  return model

In [12]:
model = build_model()

In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 1)                 70        
_________________________________________________________________
dense_1 (Dense)              (None, 10)                20        
_________________________________________________________________
dense_2 (Dense)              (None, 5)                 55        
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 6         
Total params: 151
Trainable params: 151
Non-trainable params: 0
_________________________________________________________________


In [14]:
model.fit(x_train, y_train,epochs=1000)

Epoch 1/1000
12/12 [==============================] - 1s 3ms/step - loss: 16.3152 - mae: 16.3152
Epoch 2/1000
12/12 [==============================] - 0s 2ms/step - loss: 15.7492 - mae: 15.7492
Epoch 3/1000
12/12 [==============================] - 0s 3ms/step - loss: 16.1449 - mae: 16.1449
Epoch 4/1000
12/12 [==============================] - 0s 3ms/step - loss: 15.8124 - mae: 15.8124
Epoch 5/1000
12/12 [==============================] - 0s 2ms/step - loss: 15.8047 - mae: 15.8047
Epoch 6/1000
12/12 [==============================] - 0s 2ms/step - loss: 15.8522 - mae: 15.8522
Epoch 7/1000
12/12 [==============================] - 0s 2ms/step - loss: 15.8305 - mae: 15.8305
Epoch 8/1000
12/12 [==============================] - 0s 3ms/step - loss: 15.7876 - mae: 15.7876
Epoch 9/1000
12/12 [==============================] - 0s 3ms/step - loss: 15.8462 - mae: 15.8462
Epoch 10/1000
12/12 [==============================] - 0s 3ms/step - loss: 15.9746 - mae: 15.9746
Epoch 11/1000
12/12 [========

In [16]:
score = model.evaluate(x_test, y_test, batch_size=128)

1/1 [==============================] - 0s 18ms/step - loss: 9.3295 - mae: 9.3295
